In [1]:
import pandas as pd
 
inboedel_parsed = pd.read_excel("inboedel_responses_parsed.xlsx")
reis_parsed = pd.read_excel("reis_responses_parsed.xlsx")
inboedel_gt = pd.read_excel("groundtruth_inboedel_enriched.xlsx")
reis_gt = pd.read_excel("groundtruth_reis_enriched.xlsx")

In [2]:
# === 2. Normalize GT source values for reis ===
reis_gt["source"] = reis_gt["source"].str.strip().str.lower()
reis_gt["source_normalized"] = reis_gt["source"].replace({
    "a.s.r_vp_dr_2024": "asr_2024",
    "ik_kies_zelf_(dr_2018)": "asr_ikz_2018"
})

In [15]:
# === 3. polis_versie simplification (nieuw/oud)
def simplify_polis_versie(val):
    val = str(val).lower()
    if "nieuw" in val:
        return "nieuw"
    elif "oud" in val:
        return "oud"
    else:
        return val
 
def perform_conditional_merge(parsed_df, gt_df, product_label):
    aegon_merged = pd.DataFrame()
    asr_merged = pd.DataFrame()


    # Aegon merge
    aegon_gt = gt_df[gt_df["source"].str.contains("aegon", case=False)].copy()
    aegon_gt_keys = ["vraag", "product", "dekking", "polis_versie"]
    aegon_parsed_keys = ["question", "product", "dekking", "polis_versie"]

    aegon_merged = None
    if not aegon_gt.empty:
        aegon_gt["polis_versie"] = aegon_gt["polis_versie"].apply(simplify_polis_versie)
        parsed_df["polis_versie"] = parsed_df["polis_versie"].apply(simplify_polis_versie)
 
    for col in aegon_parsed_keys:
        if col in aegon_gt.columns:
            aegon_gt[col]=aegon_gt[col].astype(str).str.strip().str.lower()
            for col in aegon_parsed_keys:
                if col in parsed_df.columns:
                   parsed_df[col] = parsed_df[col].astype(str).str.strip().str.lower()
    #for col in aegon_gt_keys:
        #aegon_gt[col] = aegon_gt[col].astype(str).str.strip().str.lower()
 
    aegon_merged = pd.merge(parsed_df, aegon_gt, left_on=aegon_parsed_keys, right_on=aegon_gt_keys, how="left")
 
    # ASR merge
    asr_gt = gt_df[gt_df["source"].str.contains("a.s.r", case=False)].copy()
    asr_gt_keys = ["vraag", "product", "dekking", "type_klant"]
    asr_parsed_keys = ["question", "product", "dekking", "type_klant"]
 
    asr_merged = None
    if col in asr_gt.columns:
            asr_gt[col]=asr_gt[col].astype(str).str.strip().str.lower()
            for col in asr_parsed_keys:
                if col in parsed_df.columns:
                   parsed_df[col] = parsed_df[col].astype(str).str.strip().str.lower()
    #if not asr_gt.epmty: 
     #for col in asr_gt_keys:
        #asr_gt[col] = asr_gt[col].astype(str).str.strip().str.lower()
     #for col in asr_parsed_keys:
        #parsed_df[col] = parsed_df[col].astype(str).str.strip().str.lower()
 
    asr_merged = pd.merge(parsed_df, asr_gt, left_on=asr_parsed_keys, right_on=asr_gt_keys, how="left")

    # Combine and save
    combined = pd.concat([aegon_merged, asr_merged], ignore_index=True)
    print(f"✅ {product_label} merged: {len(combined)} rows")
    combined.to_excel(f"{product_label.lower()}_merged_for_eval.xlsx", index=False)

In [16]:
# === Run for both products ===
perform_conditional_merge(inboedel_parsed, inboedel_gt, "inboedel")
perform_conditional_merge(reis_parsed, reis_gt, "reis")
 
print("✅ All merged files saved.")

✅ inboedel merged: 96 rows


KeyError: 'polis_versie'